## angelist.co
Scrape data using bs4 and requests

In [1]:
import re
import bs4 as bs
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np

### Part 1: Getting list of urls from "all companies" page

In [ ]:
def get_results(query, type_=None, stage=None, tech=None, signal_min=None, signal_max=None):
    
#     tech is possible only if type not None, same for others

    if type_:
        type_ = type_.replace(' ', '+')
        url = 'https://angel.co/companies' + '?company_types[]=' + type_
        if stage:
            url = url + '&stage=' + stage
            if tech:
                url = url + '&teches[]=' + tech
    if signal_min and signal_max:
        url = url + '&signal[min]='+str(signal_min)+'&signal[max]='+str(signal_max)
        score=(signal_min + signal_max)/2
    else:  
        url = 'https://angel.co/companies' 
        score = -1

    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    
#     driver = webdriver.Chrome(options=None)
    driver = webdriver.Chrome(r"C:\Users\lib-pac-kr-ppc\Desktop\3950_Final_Project\chromedriver.exe")

#     driver.set_window_position(3000,0)
    driver.get(url)
    time.sleep(5)
    
#     search_box = driver.find_element_by_class_name("search-box")
#     search_box.click()
# #     time.sleep(5)
    
#     input_bar = driver.find_element_by_class_name('keyword-input')
#     input_bar.send_keys(query)
#     input_bar.send_keys(Keys.ENTER)
    time.sleep(5)
    source = driver.page_source
    
#     while True:
#         try:
#             source = driver.page_source
#             loadMoreButton = driver.find_element_by_class_name('more').click()
#             time.sleep(1.5)
#         except: 
#             break
            
    driver.close()

    try:
        soup = bs.BeautifulSoup(source, 'lxml')
        result_list =  soup.find_all('div', {'class': 'results'})[0]
        results = result_list.find_all('div', {'data-_tn': 'companies/row'})
    except:
        print('Could not get results')
        return

    
    return results,score


def parse_results(results_score):
    df = pd.DataFrame(columns=['kind', 'link', 'name', 'pitch', 'joined', 
                               'location', 'market', 'size', 'stage', 'raised','score'],
                  index=[0])
    results=results_score[0]
    score=results_score[1]
    for result in results[1:]:
        try:
            dic = {}
            try:
                dic['kind'] = result.a['data-type']
            except: dic['kind'] = None

            try:
                dic['link'] = result.a['href']
            except: dic['link'] = None

            try:
                dic['name'] = result.a['title']
            except: dic['name'] = None
                
            try:
                dic['website'] = result.find('div', {'data-column': 'website'}).text.split()[1]
            except: dic['website'] = None

            try:
                dic['pitch'] = result.find('div', {'class': 'pitch'}).text
            except: dic['pitch'] = None

            try:
                dic['joined'] = result.find('div', {'data-column': 'joined'}).text.split('Joined')[1].strip()
            except: dic['joined'] = None

            try:
                dic['location'] = result.find('div', {'data-column': 'location'}).text.split('Location')[1].strip()
            except: dic['location'] = None

            try:
                dic['market'] = result.find('div', {'data-column': 'market'}).text.split('Market')[1].strip()
            except: dic['market'] = None

            try:
                dic['size'] = result.find('div', {'data-column': 'company_size'}).text.split()[1]
            except: dic['size'] = None

            try:
                dic['stage'] = result.find('div', {'data-column': 'stage'}).text.split('Stage')[1].strip()
            except: dic['stage'] = None

            try:
                dic['raised'] = result.find('div', {'data-column': 'raised'}).text.split('Raised')[1].strip()
            except: dic['raised'] = None
            
            try:
                dic['score'] = score
            except: dic['score'] = None
            
            df = df.append(pd.DataFrame(dic, index=[0]))

        except: pass

    df = df.reset_index(drop=True)
    return df



def get_companies(query, type_=None, stage=None, tech=None, signal_min=None, signal_max=None):
    results_score = get_results(query, type_, stage, tech, signal_min, signal_max)
    if results_score:
        df = parse_results(results_score)
        return df

In [ ]:
get_companies("Big data", type_="Startup", signal_min=8, signal_max=9)

In [ ]:
list(zip(np.arange(5.3,10,0.5),np.append(np.arange(5.7,10,0.5),[10])))

In [ ]:
list(zip(np.arange(8.5,10,0.1),np.append(np.arange(8.6,10,0.1),[10])))

In [ ]:
# queries = ['artificial intelligence', 'machine learning']
# queries = ['artificial intelligence', 'machine learning', 'data science', 'big data', 'neural network', 
#            'deep learning', 'natural language processing', 'transfer learning', 'reinforcement learning',
#            'nlp', 'recurrent neural', 'convolution neural', 'data mining', 'data extraction', 
#            'data engineering', 'web scraping', 'web scrape', 'web crawler', 'web crawling',
#            'evolutionary computation', 'genetic algorithm', 'genetic programming', 'object recognition',
#            'image understanding', 'speech processing', ' speech recognition', 'machine translation', 
#            'time series', 'ai', 'intelligent machine', 'pattern recognition', 'computer vision']

type_ = "Startup"
stages = ['Seed','Series A','Series B','Series C','Series D','Series E','Acquired']

# stages = ['Seed', 'Series A', 'Series B', 'Series C', 'Acquired']
# techs = ['Python', 'Javascript']
# techs = ['Python', 'Javascript', 'HTML5', 'Java', 'CSS', None]

scores = list(zip(np.arange(8.5,10,0.1),np.append(np.arange(8.6,10,0.1),[10])))

df = pd.DataFrame()

for score in scores:
    for stage in stages:
#         for tech in techs:
        print(type_, stage, score)

        try:
            companies = get_companies(query, type_, stage, None, score[0], score[1])
#                 companies['query'] = query
#                 companies['type_'] = type_
#                 companies['stage'] = stage
#                 companies['tech'] = tech
            print('Done')
        except Exception as e: 
            companies = pd.DataFrame()
            print('Could not get companies')
            print('\n')
            print(e)
            print('\n')
            pass

        try:
            if df.empty:
                df = companies
            else:
                df = df.append(companies)
        except:
            print('Unable to append new data.')
            pass

    df = df.reset_index(drop=True)
    df.to_csv('ai_startups.csv')
#         print(df.location.head())

            
df = df.reset_index(drop=True)
df.to_csv('ai_startups.csv')

In [ ]:
companies

### Part 1.1: Getting list of urls from "all companies" page

In [ ]:
def get_results(tech=None, market=None, stage=None):
    
#     tech is possible only if type not None, same for others


    url = 'https://angel.co/companies?company_types[]=Startup&company_types[]=Private+Company&company_types[]=SaaS&company_types[]=Mobile+App'
    if tech:
        url = url + '&teches[]=' + tech
    if market:
        market = market.replace(' ', '+')
        url = url + '&markets[]=' + market
    if stage:
        url = url + '&stage=' + stage

    url = url + '&raised[min]=1000&raised[max]=100000000000'
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    
#     driver = webdriver.Chrome(options=None)
    driver = webdriver.Chrome(r"C:\Users\lib-pac-kr-ppc\Desktop\3950_Final_Project\chromedriver.exe")

#     driver.set_window_position(3000,0)
    driver.set_window_size(5000, 1000)
    driver.get(url)
    time.sleep(5)
    
    sortby_raised = driver.find_element_by_xpath("//div[@class='column raised sortable']")
#     if sortby_raised: print("FOUND!!")
#     driver.find_element_by_class_name("sortable")
    sortby_raised.click()
    time.sleep(5)
    
#     input_bar = driver.find_element_by_class_name('keyword-input')
#     input_bar.send_keys(query)
#     input_bar.send_keys(Keys.ENTER)
#     time.sleep(5)
    source = driver.page_source
    
    while True:
        try:
            source = driver.page_source
            loadMoreButton = driver.find_element_by_class_name('more').click()
            time.sleep(3)
        except: 
            break
            
    driver.close()

    try:
        soup = bs.BeautifulSoup(source, 'lxml')
        result_list =  soup.find_all('div', {'class': 'results'})[0]
        results = result_list.find_all('div', {'data-_tn': 'companies/row'})
    except:
        print('Could not get results')
        return

    
    return results



In [ ]:
def parse_results(results):
    df = pd.DataFrame(columns=['kind', 'link', 'name', 'pitch', 'joined', 
                               'location', 'market', 'size', 'stage', 'raised'],
                  index=[0])
    
    for result in results[1:]:
        try:
            dic = {}
            try:
                dic['kind'] = result.a['data-type']
            except: dic['kind'] = None

            try:
                dic['link'] = result.a['href']
            except: dic['link'] = None

            try:
                dic['name'] = result.a['title']
            except: dic['name'] = None
                
            try:
                dic['website'] = result.find('div', {'data-column': 'website'}).text.split()[1]
            except: dic['website'] = None

            try:
                dic['pitch'] = result.find('div', {'class': 'pitch'}).text
            except: dic['pitch'] = None

            try:
                dic['joined'] = result.find('div', {'data-column': 'joined'}).text.split('Joined')[1].strip()
            except: dic['joined'] = None

            try:
                dic['location'] = result.find('div', {'data-column': 'location'}).text.split('Location')[1].strip()
            except: dic['location'] = None

            try:
                dic['market'] = result.find('div', {'data-column': 'market'}).text.split('Market')[1].strip()
            except: dic['market'] = None

            try:
                dic['size'] = result.find('div', {'data-column': 'company_size'}).text.split()[1]
            except: dic['size'] = None

            try:
                dic['stage'] = result.find('div', {'data-column': 'stage'}).text.split('Stage')[1].strip()
            except: dic['stage'] = None

            try:
                dic['raised'] = result.find('div', {'data-column': 'raised'}).text.split('Raised')[1].strip()
            except: dic['raised'] = None
            
            
            df = df.append(pd.DataFrame(dic, index=[0]))

        except: pass

    df = df.reset_index(drop=True)
    return df



def get_companies(tech=None, market=None, stage=None):
    results = get_results(tech, market, stage)
    if results:
        df = parse_results(results)
        return df

In [ ]:
get_companies(tech="HTML5", market="E-Commerce", stage=None)

In [7]:
import datetime
datetime.datetime.now()

datetime.datetime(2019, 12, 10, 16, 24, 58, 623370)

In [ ]:
filename="angelist_"+str(datetime.datetime.now())[:-7].replace(":","_").replace(" ","_")+".csv"
filename

In [ ]:
# get_companies(tech="HTML5", market="E-Commerce", stage=None)
filename="angelist_"+str(datetime.datetime.now())[:-7].replace(":","_").replace(" ","_")+".csv"

# techs = ['Python', None]
techs = ['Python', 'Javascript', 'HTML5', 'Java', 'CSS', None]

# markets = [ "Big Data"]
markets = ["E-Commerce", "Education", "Enterprise Software", "Games", "Healthcare", "Mobile", "Big Data", "AI", "ML", None]

# stages = ['Seed','Series A','Series B','Series C','Series D','Series E', 'IPO','Acquired']

df = pd.DataFrame()

for tech in techs:
    for market in markets:
        print(tech, market)

        try:
            companies = get_companies(tech, market, None)
#                 companies['query'] = query
#                 companies['type_'] = type_
#                 companies['stage'] = stage
#                 companies['tech'] = tech
            print('- Got companies')
        except Exception as e: 
            companies = pd.DataFrame()
            print('!! Could not get companies')
#             print('\n')
            print(e)
#             print('\n')
            pass

        try:
            if df.empty:
                df = companies
            else:
                df = df.append(companies)
            print("- Added {} rows".format(companies.shape[0]))
            print("- Total rows: {}".format(df.shape[0]))
        except:
            print('!! Unable to append new data.')
            pass

    df = df.reset_index(drop=True)
    df.to_csv(filename)
    print("-"*50)

            
df = df.reset_index(drop=True)
df.to_csv(filename)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df_unique = df.drop_duplicates(subset ="link" ) 

In [ ]:
df_unique.shape

In [ ]:
df_unique.sort_values("name")

In [ ]:
df_unique.to_csv('angelist_unique_4261.csv')

### Part 2: Given an angelist company link, get info from funding page

In [ ]:
url="https://angel.co/company/pebblepost"

In [2]:
def get_funding_wlogin(url,id_):

    driver = webdriver.Chrome(options=None)
    driver.get(url+"/funding")
    time.sleep(1)
    
    try:
        driver.find_element_by_partial_link_text("Log in or sign up to view")
        more_investor = True
    except:
        more_investor = False

    if more_investor:
        login_button = driver.find_element_by_link_text("Log In")
        login_button.click()
        time.sleep(1)

        email_box = driver.find_element_by_id("user_email")
        email_box.click()
        email_box.send_keys("nullbanksy@gmail.com")
        time.sleep(1)

        password_box = driver.find_element_by_id("user_password")
        password_box.click()
        password_box.send_keys("NullBanksy")
        time.sleep(1)

        login = driver.find_element_by_class_name("c-button--blue")
        login.click()
    else: pass
        
    
    while True:
        try:  
            viewMoreButton = driver.find_element_by_class_name("styles_gray__dQHNd")
            viewMoreButton.click()
            time.sleep(1.5)
#             print("ok")
        except: 
#             print("done")
            break


    source = driver.page_source
    
    try:
        soup = bs.BeautifulSoup(source, 'lxml')
        investor_list =  soup.find_all('div', {'class': 'left_4e15a'})
        bio =  soup.find_all('div', {'class': 'bio_bd676'})
    except:
        print('Could not get source')
        return
    
    driver.close()
    bio_list = [i.text for i in bio]
    name_list = [i.find("a").text for i in investor_list]
    type_list = [i.find("span").text for i in investor_list]
    
#     print(name_list)
    df = pd.DataFrame(columns=['id','name','type','bio'],
                  index=[0])
    
    for i in range(len(name_list)):
#         try:
        dic = {}
        try:
            dic['bio'] = bio_list[i]
        except: dic['bio'] = None

        try:
            dic['name'] = name_list[i]
        except: dic['name'] = None

        try:
            dic['type'] = type_list[i]
        except: dic['type'] = None

        dic['id'] = id_

        df = df.append(pd.DataFrame(dic, index=[0]))

#         except: 
#             print("error")
#             pass

    df = df.reset_index(drop=True)
    return df

In [ ]:
url2 = 'https://angel.co/company/ginx'

In [ ]:
get_funding_wlogin(url2,5)

In [2]:
comp_pd = pd.read_csv('angelist_unique_4261.csv')
comp_pd.rename(columns={'Unnamed: 0':'id'}, 
                 inplace=True)

In [4]:
comp_pd.to_csv('angelist_unique_4261_id.csv')

In [3]:
comp_pd.head()

,id,joined,kind,link,location,market,name,pitch,raised,size,stage,website
0,35,Oct ’13,Startup,https://angel.co/company/urx,San Francisco,Mobile Commerce,NaN,"\nThe Deeplink Search Engine, For Developers (...","$15,100,000",11-50,Acquired,urx.com
1,2288,May ’12,Startup,https://angel.co/company/ginx,Las Vegas,Mobile Commerce,*Ginx,\nImage based marketing and analytics \n,"$700,000",1-10,Seed,ginxapp.com
2,11708,Sep ’11,Startup,https://angel.co/company/1-page,San Francisco,SaaS,1-Page,\nNext Generation Enterprise Recruitment Proce...,"$163,000,000",11-50,IPO,1-page.com
3,11471,Apr ’12,Startup,https://angel.co/company/121nexus,Cambridge,Politics,121nexus,\nThe information platform for physical things.\n,"$794,000",1-10,-,121nexus.com
4,6887,Nov ’13,Startup,https://angel.co/company/1234enter,São Paulo,Small and Medium Businesses,1234ENTER,\nAutomated digital marketing guidance for Sma...,"$650,267",1-10,NaN,1234enter.com.br


In [ ]:
companies_input = list(zip(list(comp_pd['link']),list(comp_pd['id'])))

In [5]:
def get_investors(companies_input):
    filename="angelist_investors_"+str(datetime.datetime.now())[:-7].replace(":","_").replace(" ","_")+".csv"

    df = pd.DataFrame()
    progress=0
    for url, id_ in companies_input:
        try:
            investor_pd = get_funding_wlogin(url,id_)
            print('- Got investors')

        except Exception as e:
            investor_pd = pd.DataFrame()
            print('!! Could not get investors', id_)

        try:
            if df.empty:
                df =  investor_pd
            else:
                df = df.append(investor_pd)
            print("- Added {} rows".format(investor_pd.shape[0]))
            print("- Total rows: {}".format(df.shape[0]))

        except:
            print('!! Unable to append new data.')
            pass
        progress+=1
        print(progress, round(progress/len(companies_input)*100,2),"%")
        df = df.reset_index(drop=True)
        df.to_csv(filename)
        print("-"*50)


In [ ]:
run1= list(zip(list(comp_pd['link']),list(comp_pd['id'])))[0:214]

In [ ]:
run2= list(zip(list(comp_pd['link']),list(comp_pd['id'])))[214:1000]

In [ ]:
run3= list(zip(list(comp_pd['link']),list(comp_pd['id'])))[1000:2000]

In [6]:
run5= list(zip(list(comp_pd['link']),list(comp_pd['id'])))[3000:3420]
run6= list(zip(list(comp_pd['link']),list(comp_pd['id'])))[3420:3840]
run7= list(zip(list(comp_pd['link']),list(comp_pd['id'])))[3840:]

From `companies_input[0]` to `companies_input[213]`

In [8]:
get_investors(run5)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


- Got investors
- Added 173 rows
- Total rows: 173
1 0.24 %
--------------------------------------------------
- Got investors
- Added 70 rows
- Total rows: 243
2 0.48 %
--------------------------------------------------
- Got investors
- Added 3 rows
- Total rows: 246
3 0.71 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 247
4 0.95 %
--------------------------------------------------
- Got investors
- Added 2 rows
- Total rows: 249
5 1.19 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 250
6 1.43 %
--------------------------------------------------
- Got investors
- Added 3 rows
- Total rows: 253
7 1.67 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 254
8 1.9 %
--------------------------------------------------
- Got investors
- Added 2 rows
- Total rows: 256
9 2.14 %
--------------------------------------------------
- Got investors
-

- Got investors
- Added 1 rows
- Total rows: 614
76 18.1 %
--------------------------------------------------
- Got investors
- Added 9 rows
- Total rows: 623
77 18.33 %
--------------------------------------------------
- Got investors
- Added 3 rows
- Total rows: 626
78 18.57 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 627
79 18.81 %
--------------------------------------------------
- Got investors
- Added 4 rows
- Total rows: 631
80 19.05 %
--------------------------------------------------
- Got investors
- Added 2 rows
- Total rows: 633
81 19.29 %
--------------------------------------------------
- Got investors
- Added 5 rows
- Total rows: 638
82 19.52 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 639
83 19.76 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 640
84 20.0 %
--------------------------------------------------
- G

- Got investors
- Added 1 rows
- Total rows: 814
150 35.71 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 815
151 35.95 %
--------------------------------------------------
- Got investors
- Added 8 rows
- Total rows: 823
152 36.19 %
--------------------------------------------------
- Got investors
- Added 38 rows
- Total rows: 861
153 36.43 %
--------------------------------------------------
- Got investors
- Added 2 rows
- Total rows: 863
154 36.67 %
--------------------------------------------------
- Got investors
- Added 10 rows
- Total rows: 873
155 36.9 %
--------------------------------------------------
- Got investors
- Added 2 rows
- Total rows: 875
156 37.14 %
--------------------------------------------------
- Got investors
- Added 3 rows
- Total rows: 878
157 37.38 %
--------------------------------------------------
- Got investors
- Added 3 rows
- Total rows: 881
158 37.62 %
------------------------------------------

- Got investors
- Added 1 rows
- Total rows: 1152
223 53.1 %
--------------------------------------------------
- Got investors
- Added 10 rows
- Total rows: 1162
224 53.33 %
--------------------------------------------------
- Got investors
- Added 6 rows
- Total rows: 1168
225 53.57 %
--------------------------------------------------
- Got investors
- Added 12 rows
- Total rows: 1180
226 53.81 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 1181
227 54.05 %
--------------------------------------------------
- Got investors
- Added 10 rows
- Total rows: 1191
228 54.29 %
--------------------------------------------------
- Got investors
- Added 2 rows
- Total rows: 1193
229 54.52 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 1194
230 54.76 %
--------------------------------------------------
- Got investors
- Added 6 rows
- Total rows: 1200
231 55.0 %
---------------------------------

- Got investors
- Added 3 rows
- Total rows: 1371
296 70.48 %
--------------------------------------------------
- Got investors
- Added 3 rows
- Total rows: 1374
297 70.71 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 1375
298 70.95 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 1376
299 71.19 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 1377
300 71.43 %
--------------------------------------------------
- Got investors
- Added 10 rows
- Total rows: 1387
301 71.67 %
--------------------------------------------------
!! Could not get investors 827
- Added 0 rows
- Total rows: 1387
302 71.9 %
--------------------------------------------------
- Got investors
- Added 4 rows
- Total rows: 1391
303 72.14 %
--------------------------------------------------
- Got investors
- Added 6 rows
- Total rows: 1397
304 72.38 %
-------------------

- Got investors
- Added 1 rows
- Total rows: 1625
369 87.86 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 1626
370 88.1 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 1627
371 88.33 %
--------------------------------------------------
- Got investors
- Added 8 rows
- Total rows: 1635
372 88.57 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 1636
373 88.81 %
--------------------------------------------------
- Got investors
- Added 3 rows
- Total rows: 1639
374 89.05 %
--------------------------------------------------
- Got investors
- Added 4 rows
- Total rows: 1643
375 89.29 %
--------------------------------------------------
- Got investors
- Added 1 rows
- Total rows: 1644
376 89.52 %
--------------------------------------------------
- Got investors
- Added 6 rows
- Total rows: 1650
377 89.76 %
-----------------------------------